In [2]:
# Importing necessary libraries for the dashboard components
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import pandas as pd
from animal_shelter import AnimalShelter  # Make sure this matches your CRUD Python module file and class name

# Dashboard setup with username and password for database access
username = "Z"
password = "1234"

# Connect to the database using the CRUD module
db = AnimalShelter(username, password)

# Fetching all animal records from the database into a DataFrame
df = pd.DataFrame.from_records(db.read({}))

# Dropping the '_id' column to avoid issues with dash_table display
df.drop(columns=['_id'], inplace=True)

# Setting up the JupyterDash app
app = JupyterDash(__name__)

# Loading the company logo for inclusion in the dashboard
image_filename = '/home/zaffarshiekh_snhu/Desktop/Grazioso Salvare Logo.png'  # Path to the logo image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Including the unique identifier and logo in the app layout
app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Hr(),
    # Inserting an HTML image and unique identifier
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'10%', 'width':'10%'}),
        html.P("Created by: Zaffar Shiekh"),
    ]),
    html.Hr(),
    # RadioItems for data filtering
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
                {'label': 'Reset', 'value': 'ALL'}
            ],
            value='ALL',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),
    # Interactive data table to display animal records
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True, "hideable": True}
            for i in df.columns
        ],  # All columns are hideable - you can show/hide any of them
        data=df.to_dict('records'),
        editable=False,              # Cells are not editable
        filter_action="native",      # Enable filtering
        sort_action="native",        # Enable sorting
        sort_mode="multi",           # Allow sorting on multiple columns
        column_selectable="multi",   # Allow users to select columns
        row_selectable="multi",      # Allow users to select rows
        row_deletable=False,         # Users cannot delete rows
        selected_columns=[],         # Initially no columns are selected
        selected_rows=[],            # Initially no rows are selected
        page_action="native",        # Pagination enabled
        page_current=0,              # Start on the first page
        page_size=10,                # Number of rows per page
        style_table={'overflowX': 'auto'},  # Horizontal scrolling
        # Ensure that filtering matches text fields case insensitively
        css=[{
            'selector': '.dash-filter input',
            'rule': 'text-transform: lowercase;'
        }],
    ),
    html.Br(),
    html.Hr(),
    # Container for graph and map side-by-side
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])

# Callback to update the data table based on filter type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')])
def update_datatable(filter_type):
    # Adjusting the database query based on filter type selected by user
    query = {}  # Default to no filter (fetch all records)
    if filter_type == 'WR':
        query = {'animal_type': 'Dog', 'breed': 'Labrador Retriever'}  # Ajust as needed
    elif filter_type == 'MWR':
        query = {'animal_type': 'Dog', 'breed': 'German Shepherd'}  # Ajust as needed
    elif filter_type == 'DIT':
        query = {'animal_type': 'Dog', 'breed': 'Bloodhound'}  # Ajust as needed
    
    df_filtered = pd.DataFrame.from_records(db.read(query))
    df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

# Additional callbacks for graph and map updates go here
# (Define these callbacks to handle graph and map updates based on data table interactions)

# Running the JupyterDash server
app.run_server(debug=True)


Dash app running on http://127.0.0.1:10220/
